<center>
<img src="https://raw.githubusercontent.com/FUlyankin/ekenam_grand_research/master/images/cover.png">
</center>


# <center> Иканам гранд рисёрч </center>
## <center>  Часть седьмая: моделирование </center>


Проект **Иканам гранд рисёрч** реализуется [Иканам стьюдентс коммьюнити,](https://vk.com/ikanam)
в частности [вот этим парнем по имени Филипп.](https://vk.com/ppilif)  Если вы нашли ошибку или у вас есть предложения, замечания, деньги, слава или женщины, можно ему написать. Весь говнокод, использованный в исследовании распостраняется по лицензии [Creative Commons CC BY-NC-SA 4.0.](https://creativecommons.org/licenses/by-nc-sa/4.0/) Его можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу рисёрча.](https://github.com/FUlyankin/ekenam_grand_research) 

# 1. Отбор переменных через ~~постель~~ проверку гипотез

Проведём селекцию переменных из уже знакомой нам таблички эконометрическими способами. Делать это мы будем не в python, а в R. Потому что R делался именно для таких вещей.

In [97]:
setwd('../5. Data prepare')

In [112]:
library("dplyr")
library("ggplot2")
library("mlogit")
library('boot')
library('pROC') 

options(stringsAsFactors=FALSE)

df = read.csv('train_data_for_R.csv', sep = ';')
new_df = read.csv('test_data_for_R.csv', sep = ';')
print(dim(df))

[1] 278  60


In [113]:
head(df)

lgota,chelevoe,olimp,dogovor,ochko.zaochka,ege_diff,kozko,wall_audio_cnt,wall_audio_mean,wall_can_comment,⋯,X20pub_clust_11,X20pub_clust_12,X20pub_clust_13,X20pub_clust_14,X20pub_clust_15,X20pub_clust_16,X20pub_clust_17,X20pub_clust_18,X20pub_clust_19,target_1
0,0,1,0,0,-1.71436063,0,0.001210654,-0.37519998,1,⋯,0.02989079,0.14336749,0.03302454,0.001554972,0.04630034,0.002384977,0.03489145,0.07602706,0.07726678,1
0,0,1,0,0,0.47396584,0,0.008474576,0.40574274,1,⋯,0.04623266,0.07535394,0.09428196,0.024351069,0.03137725,0.011125820,0.12804240,0.04748053,0.02705087,1
0,0,1,0,0,0.22146663,0,0.151331719,-0.26119621,1,⋯,0.06671119,0.10695414,0.09642962,0.002388607,0.02632434,0.002383638,0.11780059,0.05199055,0.01902819,1
1,0,0,0,0,-0.07311578,0,0.000000000,-0.54773384,0,⋯,0.04488996,0.08183670,0.11842301,0.000000000,0.00967815,0.006030470,0.18226847,0.02074338,0.03613418,0
1,0,0,0,0,-1.71436063,0,0.004842615,-0.03013227,1,⋯,0.01079370,0.07079418,0.05716315,0.020233241,0.02941722,0.005071339,0.21965020,0.02719962,0.05738306,1
1,0,0,0,0,-2.09310944,0,0.228813559,-0.11217419,1,⋯,0.03317695,0.07697521,0.09042338,0.000000000,0.09171961,0.001280769,0.20409107,0.02315552,0.07277238,1


Оцениваем логистическую регрессию! 

In [114]:
m_logit$family


Family: binomial 
Link function: logit 


In [115]:
m_logit <- glm(data = df, target_1~. ,family=binomial(link="logit"))
summary(m_logit)

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


Call:
glm(formula = target_1 ~ ., family = binomial(link = "logit"), 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2684  -0.4433   0.2302   0.6281   1.9658  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)  
(Intercept)               2.12676    1.31746   1.614   0.1065  
lgota                     1.37421    1.03523   1.327   0.1844  
chelevoe                 -0.42046    1.13237  -0.371   0.7104  
olimp                     1.04249    1.38651   0.752   0.4521  
dogovor                   0.09935    0.66415   0.150   0.8811  
ochko.zaochka             1.95447    0.94139   2.076   0.0379 *
ege_diff                  0.54809    0.30038   1.825   0.0680 .
kozko                    -0.18549    0.41800  -0.444   0.6572  
wall_audio_cnt           -0.87474    4.50712  -0.194   0.8461  
wall_audio_mean           0.11239    0.17481   0.643   0.5203  
wall_can_comment          0.99555    0.59517   1.673   0.0944 .
wall_comment_cnt 

Отбираем только значимые на $10\%$ уровне переменные. 

In [116]:
res = summary(m_logit)
n = length(res$coefficients[,4])
df = df[,c(res$coefficients[,4][2:n] < 0.1, TRUE)]
dim(df)
#head(df)

m_logit_1 <- glm(data = df, target_1~. ,family=binomial(link="logit"),x=TRUE)
res_1 = summary(m_logit_1)
lrtest(m_logit,m_logit_1)
res_1

[1] 278  12

#Df,LogLik,Df,Chisq,Pr(>Chisq)
60,-104.3160,NA,NA,NA
12,-132.0627,-48,55.49342,0.2131512



Call:
glm(formula = target_1 ~ ., family = binomial(link = "logit"), 
    data = df, x = TRUE)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2832  -0.8916   0.4667   0.7509   1.6632  

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)   
(Intercept)           -0.12140    0.43967  -0.276  0.78245   
ochko.zaochka          1.08030    0.61512   1.756  0.07905 . 
ege_diff               0.36458    0.15035   2.425  0.01531 * 
wall_can_comment       1.13315    0.38509   2.943  0.00326 **
wall_comment_max       1.31841    0.74632   1.767  0.07730 . 
wall_like_mean        -0.06018    0.13290  -0.453  0.65068   
wall_poll_cnt          7.98564    4.93712   1.617  0.10578   
wall_post_android_cnt -4.69104    1.98783  -2.360  0.01828 * 
wall_post_iphone_cnt  -6.45991    3.01081  -2.146  0.03191 * 
wall_video_mean        0.40984    0.19552   2.096  0.03607 * 
X20pub_clust_0         6.28098    3.24856   1.933  0.05318 . 
X20pub_clust_3        17.88452 

In [117]:
n = length(res_1$coefficients[,4])
df = df[,c(res_1$coefficients[,4][2:n] < 0.1, TRUE)]
dim(df)
# head(df)

m_logit_2 <- glm(data = df, target_1~., family=binomial(link="logit"))
res_2 = summary(m_logit_2)
lrtest(m_logit_1,m_logit_2)
res_2

[1] 278  10

#Df,LogLik,Df,Chisq,Pr(>Chisq)
12,-132.0627,NA,NA,NA
10,-134.2570,-2,4.38853,0.1114404



Call:
glm(formula = target_1 ~ ., family = binomial(link = "logit"), 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2776  -0.8138   0.4774   0.7597   1.6885  

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)   
(Intercept)           -0.09794    0.43917  -0.223  0.82352   
ochko.zaochka          0.95084    0.60697   1.567  0.11722   
ege_diff               0.38037    0.14834   2.564  0.01034 * 
wall_can_comment       1.15838    0.38725   2.991  0.00278 **
wall_comment_max       1.34265    0.76095   1.764  0.07766 . 
wall_post_android_cnt -2.10978    1.33331  -1.582  0.11357   
wall_post_iphone_cnt  -5.25849    2.73821  -1.920  0.05481 . 
wall_video_mean        0.43372    0.20031   2.165  0.03037 * 
X20pub_clust_0         6.25255    3.21392   1.945  0.05172 . 
X20pub_clust_3        17.36287    8.74068   1.986  0.04698 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial 

In [118]:
n = length(res_2$coefficients[,4])
df = df[,c(res_2$coefficients[,4][2:n] < 0.1, TRUE)]
dim(df)
#head(df)

m_logit_3 <- glm(data = df, target_1~., family=binomial(link="logit"))
res_3 = summary(m_logit_3)
lrtest(m_logit_2,m_logit_3)

res_3

[1] 278   8

#Df,LogLik,Df,Chisq,Pr(>Chisq)
10,-134.2570,NA,NA,NA
8,-136.7087,-2,4.903398,0.08614709



Call:
glm(formula = target_1 ~ ., family = binomial(link = "logit"), 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2736  -0.8836   0.5039   0.7580   1.6266  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)   
(Intercept)          -0.06925    0.42976  -0.161  0.87198   
ege_diff              0.34212    0.14473   2.364  0.01809 * 
wall_can_comment      1.06443    0.37665   2.826  0.00471 **
wall_comment_max      1.25442    0.74842   1.676  0.09372 . 
wall_post_iphone_cnt -5.37323    2.74146  -1.960  0.05000 * 
wall_video_mean       0.39667    0.19328   2.052  0.04014 * 
X20pub_clust_0        6.46230    3.12955   2.065  0.03893 * 
X20pub_clust_3       16.25065    8.37413   1.941  0.05231 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 324.17  on 277  degrees of freedom
Residual deviance: 273.42  on 270  degrees of freedom
AI

In [105]:
n = length(res_3$coefficients[,4])
df = df[,c(res_3$coefficients[,4][2:n] < 0.1, TRUE)]
dim(df)
#head(df)

m_logit_4 <- glm(data = df, target_1~., family=binomial(link="logit"))
res_4 = summary(m_logit_4)
lrtest(m_logit_3,m_logit_4)

res_4

[1] 278  22

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

#Df,LogLik,Df,Chisq,Pr(>Chisq)
22,-102.7236,NA,NA,NA
22,-102.7236,0,2.842171e-14,0



Call:
glm(formula = target_1 ~ ., family = binomial(link = "logit"), 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.4824  -0.3765   0.1758   0.6730   1.8876  

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)             2.2075     0.8011   2.756 0.005857 ** 
profile_folowers_cnt   -5.9695     2.3593  -2.530 0.011399 *  
profile_skype_dummy    -1.6506     0.6874  -2.401 0.016335 *  
profile_albums_cnt     20.1929     4.6351   4.357 1.32e-05 ***
profile_audio_cnt      -3.7783     1.5176  -2.490 0.012786 *  
profile_lang_cnt        4.6777     1.5188   3.080 0.002071 ** 
wall_comment_max        5.5715     1.6627   3.351 0.000806 ***
wall_like_mean          0.4679     0.2474   1.891 0.058620 .  
wall_link_cnt         -44.7819    11.0768  -4.043 5.28e-05 ***
wall_post_cnt         -13.9129     4.5971  -3.026 0.002474 ** 
wall_post_ipad_cnt     13.8106     3.6504   3.783 0.000155 ***
wall_repost_mean       -1.025

In [106]:
pred_df = new_df[colnames(t(res_4$coefficients))[-1]]
pred_prob = predict(m_logit_4, pred_df, se=TRUE)
real_targets = new_df['target_1']

In [111]:
pred_df = data_frame(prob = plogis(pred_prob$fit),
                     left_ci=plogis(pred_prob$fit-1.96*pred_prob$se.fit),
                     right_ci=plogis(pred_prob$fit+1.96*pred_prob$se.fit))
head(pred_df)

t = data.frame(prob = pred_prob$fit, target = real_targets)
roc.data <- roc(t$target,t$prob)
roc.data$auc

ric_auc_score <- function(x,y){
    roc.data <- roc(x,y)
    return(roc.data$auc)
}

kfCV <- cv.glm(data=df, glmfit=m_logit_4, K=5, cost = ric_auc_score)
kfCV$delta

prob,left_ci,right_ci
0.99102978,0.915680935,0.9991111
0.82728916,0.674360438,0.9172152
0.80408672,0.685302233,0.8855249
0.99953006,0.989051912,0.9999800
0.05827789,0.006018485,0.3874386
0.72705933,0.455752869,0.8944439


Area under the curve: 0.472

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”

[1] 0.7931529 0.8053937

In [ ]:
# ROC - кривая
# Можно установить любой порог отсечение - погиб, выжил.
# Этот выбор влияет на вероятности разных ошибок.
pr_t <- predict(m_logit,t,se=TRUE)
t <- cbind(t,pr_t)
#превратим оценки в вероятности
t <- mutate(t,prob=plogis(fit))
select(t,age,survived,prob)
#таблица вероятностей выжить и факты

#Теперь на основании этих данных мы переберем пороги отсечения
# и построим ROC-curve.
roc.data <- roc(t$prob,t$survived)
str(roc.data)
#список из 3 переменных cutoffs - пороги
#fpк - доля ошибочно классифицированных не выживших
#tpr - доля правильно выживших пассажиров

qplot(x=roc.data$cutoffs,y=roc.data$tpr,geom="line")
#по горизонтали - пороги по вертикали - доля верно выживших
# если порог низок, большое кол-во выживет по прогнозу и доля высока
# если порог высок, мало выживших выживет.

qplot(x=roc.data$cutoffs,y=roc.data$fpr,geom="line")
# при маленьком пороге много неверно невыживших 
# при большом пороге много верно невыживших 

#Roc кривой называется сочетание этих нрафиков на одном

qplot(x=roc.data$fpr,y=roc.data$tpr,geom="line")

# дилема выбора между верно погибшими и верно выжившими
# если порог высок то у меня низка доля верно выживших 
# но высока доля верно невыживших и наоборот при низком 
# пороге.

# Надо как-то выбрать порог....


In [ ]:
#надо указать R что ряд текстовых переменных факторные
t <- mutate(t,sex=as.factor(sex),pclass=as.factor(pclass),survived=as.factor(survived))
#описательные статистики по каждой переменной:
summary(t)

#Имеет смысл для качественной переменной построить несколько
#типов графиков.

mosaic(data=t,~sex+pclass+survived,shade=TRUE)
#на графике видны соотношения между всеми факторами.
# про независимость признаков и закраску не понял

#построим график виалончель
qplot(data=t,x=survived,y=age,geom="violin")

#если поделить пассажиров на выживших и не выживших
#и отдожить по вертикали количественную переменную
#а именно возраст, то мы видим что возраст меняется от
# 0 до 80 и среди тех и других по возрастам распределение
# примерно одинаковое.

#раньше вместо виолнчели строли ящик с усами. 

qplot(data=t,x=survived,y=age,geom="boxplot")

#видно то же самое ;)

# Первые два варианта (виол и усы) показывают лишь распределение выживших и не выживших,
# а не их количество, на графике-мозаике же сравнить можно, причём особенно
# наглядно это будет, если указать переменную survived первой.

#Построим плотности выживших и невыживших на одном графике. 
qplot(data=t,x=age,y=..count..,fill=survived,geom="density",position="stack")
#                 количество    закраска                 как расположены     
#                 наблюдений    в соотв.                 накапливаются по   
#                 комп сам      выжил или нет            вертикали

qplot(data=t,x=age,y=..count..,fill=survived,geom="density",position="fill")
#сглаженная услтвная вероятность. Среди пожилого возрата большая састь погибла и т.д.
#график илюстрирует только соотношение, но не количество.




#оценка logit и probit моделей.

m_logit <- glm(data=t, survived~sex+age+pclass+fare,
               family=binomial(link="logit"),x=TRUE)

m_probit <- glm(data=t, survived~sex+age+pclass+fare,
               family=binomial(link="probit"),x=TRUE)

summary(m_logit)
summary(m_probit)


# 
# ИНТЕРПРЕТАЦИЯ!!!
#
#

vcov(m_logit)
#выдаст ковариац матрицу

#коэфициены интерпретируются плохо. Лучше интерпретировать
#вероятности и предельные эффекты
#построим прогноз

newdata <- data.frame(age=seq(5,100,length=100),sex="male",pclass="2nd",fare=100)
head(newdata)
pr_logit <- predict(m_logit,newdata,se=TRUE)
newdata_pr <- cbind(newdata,pr_logit)
#cbind объединяет столбики

head(newdata_pr)
#переменная fit - предсказанная y* residual.scale - вспомогательная 

#Доверительный интервал для вероятности и LR тест в R
newdata_pr <- mutate(newdata_pr,prob=plogis(fit),
                     left_ci=plogis(fit-1.96*se.fit),
                     right_ci=plogis(fit+1.96*se.fit))
head(newdata_pr)

#Проилюстрируем графически
qplot(data=newdata_pr, x=age,y=prob,geom="line")
#видим, что при росте возраста вероятность выжить падает
qplot(data=newdata_pr, x=age,y=prob,geom="line")+
  geom_ribbon(aes(ymin=left_ci,ymax=right_ci),alpha=0.2)
#добавили доверительные интервалы. Точность прогнозирования
# увеличивается с возрастом, т.к. мало детей


#Сравним нашу модель с моделью без финансовых переменных
#первая модель оценивалась не по всей выборке. R откидывает пропущенное
t2 <- select(t,sex,age,pclass,survived,fare) %>% na.omit
m_logit2 <- glm(data=t2,survived~sex+age,
                family=binomial(link="logit"),x=TRUE)
lrtest(m_logit,m_logit2)
#гипотеза о том что финансовые переменные не оказывают
#влияние на вер выжить отвергнута так как pvalue мало.



#Предельные эффекты в R
maBina(m_logit)
#команда даст предельные эффекты для идивида 
#среднего возраста среднего пола и т.п. 

maBina(m_logit,x.mean=FALSE)
#дает на сколько увеличение возраста на 1 год для каждого
#пассажира увеличит или уменьшит его вер выжить а после
#она усредняется по всем пассажирам

#Проинтерпретируем предельные эффекты. 



#сделаем вид что мы не знаем о логит и пробит и применим мнк
# ЭТО НЕКОРРЕКТНО! 

m_ols <- lm(data=t, as.numeric(survived)~sex+age+pclass+fare)
summary(m_ols)
#предскажем вероятности по ней
pr_ols <- predict(m_ols,newdata)
head(pr_ols)
# вероятности больше 1!!!




# ROC - кривая
# Можно установить любой порог отсечение - погиб, выжил.
# Этот выбор влияет на вероятности разных ошибок.
pr_t <- predict(m_logit,t,se=TRUE)
t <- cbind(t,pr_t)
#превратим оценки в вероятности
t <- mutate(t,prob=plogis(fit))
select(t,age,survived,prob)
#таблица вероятностей выжить и факты

#Теперь на основании этих данных мы переберем пороги отсечения
# и построим ROC-curve.
roc.data <- roc(t$prob,t$survived)
str(roc.data)
#список из 3 переменных cutoffs - пороги
#fpк - доля ошибочно классифицированных не выживших
#tpr - доля правильно выживших пассажиров

qplot(x=roc.data$cutoffs,y=roc.data$tpr,geom="line")
#по горизонтали - пороги по вертикали - доля верно выживших
# если порог низок, большое кол-во выживет по прогнозу и доля высока
# если порог высок, мало выживших выживет.

qplot(x=roc.data$cutoffs,y=roc.data$fpr,geom="line")
# при маленьком пороге много неверно невыживших 
# при большом пороге много верно невыживших 

#Roc кривой называется сочетание этих нрафиков на одном

qplot(x=roc.data$fpr,y=roc.data$tpr,geom="line")

# дилема выбора между верно погибшими и верно выжившими
# если порог высок то у меня низка доля верно выживших 
# но высока доля верно невыживших и наоборот при низком 
# пороге.

# Надо как-то выбрать порог....
